# 3. KPI Evaluation & Visualisation
Batch inference = batch allocation decisions. Compute throughput, idle-time reduction, decision latency.

In [ ]:
import numpy as np
import pandas as pd
import time

# Simulated predictions (replace with real model + predict)
probs = np.array([[0.1,0.7,0.2],[0.4,0.5,0.1],[0.2,0.1,0.7]])
allocations = np.argmax(probs, axis=1)
allocations

### Inference / Decision Step (refactored from predict.py)

In [ ]:
"""
Predict flower name from an image with predict.py along with the probability of that name. That is, you'll pass
in a single image /path/to/image and return the flower name and class probability.

Basic usage: python predict.py /path/to/image checkpoint
Options:
Return top-k most likely classes:               python predict.py input checkpoint --top_k 3
Use a mapping of categories to real names:      python predict.py input checkpoint --category_names cat_to_name.json
Use GPU for inference:                          python predict.py input checkpoint --gpu
"""

# TODO: IMPORTS HERE
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import torch
import argparse
import utility as util
import json

#######################################################################################################################

# TODO: Add command line arguments ######

# Set DEFAULT values for all future arguments
default_checkpoint = 'checkpoint.pth'
default_cat_name_file = 'cat_to_name.json'
default_image_path = 'flowers/test/10/image_07104.jpg'
default_top_k = 5
default_arch = 'densenet161'
default_device = 'gpu'

# Creates Argument Parser object called parser
parser = argparse.ArgumentParser()

# Arguments for inputs
parser.add_argument('--checkpoint', type=str, default=default_checkpoint, help='Name of checkpoint of model for '
                                                                               'prediction')
parser.add_argument('--arch', type=str, default=default_arch, help='Choose between "densenet161" or "alexnet"')
parser.add_argument('--dir', type=str, default=default_image_path, help='File path of image to predict')
parser.add_argument('--top_k', type=int, default=default_top_k, help='Number of classes in the prediction output')
parser.add_argument('--json', type=str, default=default_cat_name_file, help='File with names of categories')
parser.add_argument('--gpu', type=str, default=default_device, help='Uses "GPU" if available')

# Assign variable in parse_args() to access the arguments in the argparse object
args = parser.parse_args()

# Assign the variables to be used in the rest of this script
checkpoint = args.checkpoint
cat_name_file = args.json
image_path = args.dir
top_k = args.top_k
arch = args.arch
device = args.gpu

#######################################################################################################################

# TODO: Open the category to name file json file
with open(cat_name_file, 'r') as f:
    cat_to_name = json.load(f)


#######################################################################################################################
# TODO: Load Model/Checkpoint

def load_checkpoint(filepath):
    """
    Inputs: filepath to the checkpoint (.pth) extension
    Outputs: model.state_dict -- model architecture info, includes parameter matrices for each of the layers
             optimizer -- optimizer parameters
             model.class_to_idx --
    """

    # Load the checkpoint
    checkpoint_load = torch.load(filepath)

    # Load model architecture and parameters
    model.classifier = checkpoint_load['classifier']
    model.load_state_dict(checkpoint_load['state_dict'])
    model.class_to_idx = checkpoint_load['class_to_idx']

    # Load hyper-parameters (optional)
    epochs = checkpoint_load['epochs']
    learning_rate = checkpoint_load['learning_rate']

    # Load other relevant information (optional)
    network = checkpoint_load['network']
    input_size = checkpoint_load['input_size']
    output_size = checkpoint_load['output_size']
    hidden_units = checkpoint_load['hidden_units']
    optimizer = checkpoint_load['optimizer']

    print(f"Model - Classifier: {model.classifier}\n"
          f"Learning rate: {learning_rate}\n"
          f"Network: {network}\n"
          f"Epochs: {epochs}\n"
          f"Input size: {input_size}\n"
          f"Output size: {output_size}\n"
          f"Hidden layers: {hidden_units}\n"
          f"Optimizer: {optimizer}\n")

    return model


# DEFINE THE MODEL DURING THE LOAD
model = load_checkpoint(checkpoint)


#######################################################################################################################
# TODO: Define prediction function

def predict(fn_image_path, fn_model, fn_topk):
    """ Predict the class (or classes) of an image using a trained deep learning model.
        Inputs: image_path, model, topk (default=5)
        Outputs: top-k probabilities and top-k classes
    """

    # TODO: Implement the code to predict the class from an image file

    # Use GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Process any input image using pre-defined function: process_image(image_path)
    image = util.process_image(image_path)

    # to accommodate RuntimeError error
    image = image.unsqueeze(0)

    # Move image to 'device'
    image = image.to(device)

    # Predict the image
    # Calculate log(Softmax) -> Convert to Prob distribution
    log_ps = model(image)
    ps = torch.exp(log_ps)

    # Get top-k probabilities
    top_prob, top_class = ps.topk(k=top_k, dim=1)

    # Move top prob to cpu (vs cuda), then numpy (vs tensor), then into a list
    top_prob = top_prob.detach().cpu().numpy().tolist()[0]

    # Convert top labels to cpu (vs cuda), then numpy (vs tensor), then into a list
    top_class = top_class.detach().cpu().numpy()[0]

    # Invert class_to_idx --> idx_to_class and save to dictionary
    idx_to_class = {idx: cls for cls, idx in model.class_to_idx.items()}

    # Assign labels to the top_class
    class_labels = [idx_to_class[cls] for cls in top_class]

    flowers = [cat_to_name[label] for label in class_labels]

    # return probability of top_k and their corresponding class names
    return top_prob, class_labels, flowers


# TODO: Display topK with their probabilities
prob, classes, flower_names = predict(fn_image_path=image_path, fn_

### KPI Functions (refactored from calculates_results_stats.py)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# */AIPND-revision/intropyproject-classify-pet-images/calculates_results_stats.py
#                                                                             
# PROGRAMMER:
# DATE CREATED:                                  
# REVISED DATE: 
# PURPOSE: Create a function calculates_results_stats that calculates the 
#          statistics of the results of the programrun using the classifier's model 
#          architecture to classify the images. This function will use the 
#          results in the results dictionary to calculate these statistics. 
#          This function will then put the results statistics in a dictionary
#          (results_stats_dic) that's created and returned by this function.
#          This will allow the user of the program to determine the 'best' 
#          model for classifying the images. The statistics that are calculated
#          will be counts and percentages. Please see "Intro to Python - Project
#          classifying Images - xx Calculating Results" for details on the 
#          how to calculate the counts and percentages for this function.    
#         This function inputs:
#            -The results dictionary as results_dic within calculates_results_stats 
#             function and results for the function call within main.
#         This function creates and returns the Results Statistics Dictionary -
#          results_stats_dic. This dictionary contains the results statistics 
#          (either a percentage or a count) where the key is the statistic's 
#           name (starting with 'pct' for percentage or 'n' for count) and value 
#          is the statistic's value.  This dictionary should contain the 
#          following keys:
#            n_images - number of images
#            n_dogs_img - number of dog images
#            n_notdogs_img - number of NON-dog images
#            n_match - number of matches between pet & classifier labels
#            n_correct_dogs - number of correctly classified dog images
#            n_correct_notdogs - number of correctly classified NON-dog images
#            n_correct_breed - number of correctly classified dog breeds
#            pct_match - percentage of correct matches
#            pct_correct_dogs - percentage of correctly classified dogs
#            pct_correct_breed - percentage of correctly classified dog breeds
#            pct_correct_notdogs - percentage of correctly classified NON-dogs
#
##
# TODO 5: Define calculates_results_stats function below, please be certain to replace None
#       in the return statement with the results_stats_dic dictionary that you create 
#       with this function
# 
def calculates_results_stats(results_dic):
    """
    Calculates statistics of the results of the program run using classifier's model 
    architecture to classifying pet images. Then puts the results statistics in a 
    dictionary (results_stats_dic) so that it's returned for printing as to help
    the user to determine the 'best' model for classifying images. Note that 
    the statistics calculated as the results are either percentages or counts.
    Parameters:
      results_dic - Dictionary with key as image filename and value as a List 
             (index)idx 0 = pet image label (string)
                    idx 1 = classifier label (string)
                    idx 2 = 1/0 (int)  where 1 = match between pet image and 
                            classifer labels and 0 = no match between labels
                    idx 3 = 1/0 (int)  where 1 = pet image 'is-a' dog and 
                            0 = pet Image 'is-NOT-a' dog. 
                    idx 4 = 1/0 (int)  where 1 = Classifier classifies image 
                            'as-a' dog and 0 = Classifier classifies image  
                            'as-NOT-a' dog.
    Returns:
     results_stats_dic - Dictionary that contains the results statistics (either
                    a percentage or a count) where the key is the statistic's 
                     name (starting with 'pct' for percentage or 'n' for count)
                     and the value is the statistic's value. See comments above
                     and the previous topic Calculating Results in the class for details
                     on how to calculate the counts and statistics.
    """        
    
     # Creates empty dictionary for results_stats_dic
    results_stats_dic = dict()
    
    # Sets all counters to initial values of zero so that they can 
    # be incremented while processing through the images in results_dic 
    results_stats_dic['n_dogs_img'] = 0
    results_stats_dic['n_match'] = 0
    results_stats_dic['n_correct_dogs'] = 0
    results_stats_dic['n_correct_notdogs'] = 0
    results_stats_dic['n_correct_breed'] = 0       
    
    # process through the results dictionary
    for key in results_dic:
         
        # Labels Match Exactly
        if results_dic[key][2] == 1:
            results_stats_dic['n_match'] += 1

        # TODO: 5a. REPLACE pass with CODE that counts how many pet images of
        #           dogs had their breed correctly classified. This happens 
        #           when the pet image label indicates the image is-a-dog AND 
        #           the pet image label and the classifier label match. You 
        #           will need to write a conditional statement that determines
        #           when the dog breed is correctly classified and then 
        #           increments 'n_correct_breed' by 1. Recall 'n_correct_breed' 
        #           is a key in the results_stats_dic dictionary with it's value 
        #           representing the number of correctly classified dog breeds.
        #           
        # Pet Image Label is a Dog AND Labels match- counts Correct Breed
        
        
        if results_dic[key][2] == 1 and results_dic[key][3] == 1:
            results_stats_dic['n_correct_breed'] += 1
    
    
        
        # Pet Image Label is a Dog - counts number of dog images
        if results_dic[key][3] == 1:
            results_stats_dic['n_dogs_img'] += 1
            
            # Classifier classifies image as Dog (& pet image is a dog)
            # counts number of correct dog classifications
            if results_dic[key][4] == 1:
                results_stats_dic['n_correct_dogs'] += 1

        # TODO: 5b. REPLACE pass with CODE that counts how many pet images 
        #           that are NOT dogs were correctly classified. This happens 
        #           when the pet image label indicates the image is-NOT-a-dog 
        #           AND the classifier label indicates the images is-NOT-a-dog.
        #           You will need to write a conditional statement that 
        #           determines when the classifier label indicates the image 
        #           is-NOT-a-dog and then increments 'n_correct_notdogs' by 1. 
        #           Recall the 'else:' above 'pass' already indicates that the 
        #           pet image label indicates the image is-NOT-a-dog and 
        #          'n_correct_notdogs' is a key in the results_stats_dic dictionary 
        #           with it's value representing the number of correctly 
        #           classified NOT-a-dog images.
        #           
        # Pet Image Label is NOT a Dog
        #if results_dic[key][3] == 0:
            #results_stats_dic['n_correct_notdogs'] += 1
        
        
        else:
            # Classifier classifies image as NOT a Dog(& pet image isn't a dog)
            # counts number of correct NOT dog clasifications.
            if results_dic[key][3]==0 and results_dic[key][4]==0:
                results_stats_dic['n_correct_notdogs'] += 1
    
                    


    # Calculates run statistics (counts & percentages) below that are calculated
    # using the counters from above.
    
    # calculates number of total images
    results_stats_dic['n_images'] = len(results_dic)

    # calculates number of not-a-d

### Visualisation placeholders
Export your Matplotlib charts to `../visuals/performance_comparison.png` and `../visuals/allocation_heatmap.png`.